In [8]:
from pycaret.datasets import get_data

In [11]:
dataset = get_data('diamond')

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
0,1.10,Ideal,H,SI1,VG,EX,GIA,5169
1,0.83,Ideal,H,VS1,ID,ID,AGSL,3470
2,0.85,Ideal,H,SI1,EX,EX,GIA,3183
3,0.91,Ideal,E,SI1,VG,VG,GIA,4370
4,0.83,Ideal,G,SI1,EX,EX,GIA,3171


In [12]:
#check the shape of data
dataset.shape

(6000, 8)

In [14]:
#check general info of data
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Carat Weight  6000 non-null   float64
 1   Cut           6000 non-null   object 
 2   Color         6000 non-null   object 
 3   Clarity       6000 non-null   object 
 4   Polish        6000 non-null   object 
 5   Symmetry      6000 non-null   object 
 6   Report        6000 non-null   object 
 7   Price         6000 non-null   int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 375.1+ KB


In [16]:
#from our data we can see that we need to predict price
#regression is good here, couse its continuous type of data

In [17]:
#we can check several models to compare their results and choose the best one

Data preparing

In [18]:
#do the train-test split to check seen and unseen data
data = dataset.sample(frac=0.9, random_state=786).reset_index(drop=True)
data_unseen = dataset.drop(data.index).reset_index(drop=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (5400, 8)
Unseen Data For Predictions: (600, 8)


In [19]:
from pycaret.regression import *

# setup and normalize our data
reg = setup(data = data, target = 'Price', session_id=123,
                  normalize = True, experiment_name = 'diamond1')

,Description,Value
0,Session id,123
1,Target,Price
2,Target type,Regression
3,Original data shape,"(5400, 8)"
4,Transformed data shape,"(5400, 29)"
5,Transformed train set shape,"(3779, 29)"
6,Transformed test set shape,"(1621, 29)"
7,Ordinal features,1
8,Numeric features,1
9,Categorical features,6


In [20]:
top_models = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,758.7793,2661557.6112,1584.1538,0.9743,0.0820,0.0608,0.2270
rf,Random Forest Regressor,761.3311,2951077.3659,1665.3939,0.9714,0.0816,0.0597,0.2110
lightgbm,Light Gradient Boosting Machine,752.6246,3056225.3710,1687.9572,0.9711,0.0773,0.0567,0.0660
gbr,Gradient Boosting Regressor,919.9681,3758041.1316,1899.6475,0.9633,0.1024,0.0770,0.0970
dt,Decision Tree Regressor,1012.6874,5386864.0817,2245.0469,0.9465,0.1083,0.0780,0.0450
ridge,Ridge Regression,2413.7360,14243970.9512,3744.2518,0.8607,0.6687,0.2869,0.0440
llar,Lasso Least Angle Regression,2413.3101,14243412.3661,3744.1744,0.8607,0.6720,0.2868,0.0450
br,Bayesian Ridge,2411.8365,14243532.4236,3744.1058,0.8607,0.6639,0.2865,0.0440
lasso,Lasso Regression,2413.3101,14243399.7719,3744.1729,0.8607,0.6720,0.2868,0.1570
lr,Linear Regression,2417.0918,14236866.4983,3744.0036,0.8607,0.6719,0.2875,0.2670


In [23]:
print(top_models)

ExtraTreesRegressor(n_jobs=-1, random_state=123)


In [24]:
knn = create_model('knn')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3404.4698,30619432.7951,5533.4829,0.6638,0.4029,0.3161
1,3658.2608,47311849.9368,6878.3610,0.6374,0.4146,0.3257
2,3154.6577,39707001.3502,6301.3492,0.6358,0.3920,0.2980
3,3314.9164,34336057.6800,5859.6977,0.6212,0.4183,0.2990
4,3134.1545,28567323.3985,5344.8408,0.6437,0.3864,0.3037
5,2973.5751,36761843.9800,6063.1546,0.6862,0.3726,0.2880
6,3363.0614,38067628.4195,6169.8970,0.6424,0.3987,0.2978
7,3161.0095,28748088.6584,5361.7244,0.6364,0.4141,0.3139
8,3273.2233,33613172.2152,5797.6868,0.6699,0.3728,0.2761


In [25]:
print(knn)

KNeighborsRegressor(n_jobs=-1)


In [26]:
tuned_knn = tune_model(knn)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2233.5237,18891437.6792,4346.4282,0.7926,0.2910,0.1962
1,2793.4258,38972026.7640,6242.7579,0.7013,0.3200,0.2191
2,2111.0042,26286490.5477,5127.0353,0.7589,0.2815,0.1870
3,2312.6637,22290724.4922,4721.3054,0.7541,0.3035,0.1834
4,2281.1395,19714253.1063,4440.0735,0.7541,0.3142,0.2207
5,2196.1473,28935087.4191,5379.1344,0.7530,0.2761,0.1863
6,2448.5489,28427909.1307,5331.7829,0.7329,0.2806,0.1870
7,2236.5871,20772847.4012,4557.7239,0.7373,0.3206,0.2074
8,2263.1548,21475972.7926,4634.2176,0.7891,0.2689,0.1717


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [27]:
# after we tuned model, results get better from 0.64 to 0.75

In [28]:
plot_model(tuned_knn, plot = 'parameter')

,Parameters
algorithm,auto
leaf_size,30
metric,minkowski
metric_params,None
n_jobs,-1
n_neighbors,9
p,2
weights,distance


In [34]:
#Decision Tree
dt = create_model('dt', fold = 5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,983.9173,4624019.5021,2150.3533,0.9583,0.1104,0.0782
1,978.9878,3302395.6938,1817.2495,0.9669,0.1017,0.0770
2,1065.9009,6973191.8237,2640.6802,0.9294,0.1176,0.0825
3,1019.3413,4582632.8975,2140.7085,0.9507,0.1081,0.0793
4,1005.3715,4945902.2156,2223.9384,0.9522,0.1123,0.0787
Mean,1010.7038,4885628.4265,2194.5860,0.9515,0.1100,0.0791
Std,31.2192,1185615.8967,263.4782,0.0124,0.0052,0.0018


In [32]:
tuned_dt = tune_model(dt)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,999.8699,2893945.9579,1701.1602,0.9682,0.1072,0.0826
1,1087.1638,6720052.1072,2592.3063,0.9485,0.1059,0.0819
2,1002.3163,3276189.6684,1810.0248,0.9700,0.1052,0.0812
3,1079.5332,4039392.6785,2009.8240,0.9554,0.1173,0.0869
4,1100.0676,7883046.7345,2807.6764,0.9017,0.1189,0.0841
5,1274.8251,11019979.7098,3319.6355,0.9059,0.1248,0.0894
6,1071.7476,4472202.5033,2114.7583,0.9580,0.1084,0.0813
7,980.7362,3286501.5489,1812.8711,0.9584,0.1101,0.0809
8,1101.9207,4441966.3616,2107.5973,0.9564,0.1114,0.0873


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [35]:
#Random Forest
rf = create_model('rf', round = 2)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,681.68,1224394.65,1106.52,0.99,0.08,0.06
1,808.81,5016363.59,2239.72,0.96,0.08,0.06
2,734.11,1693231.21,1301.24,0.98,0.08,0.06
3,748.64,1585590.22,1259.20,0.98,0.08,0.06
4,745.56,3673407.87,1916.61,0.95,0.09,0.06
5,871.43,5529579.92,2351.51,0.95,0.09,0.06
6,730.04,2589214.10,1609.10,0.98,0.08,0.06
7,712.53,1736124.51,1317.62,0.98,0.08,0.06
8,731.01,2248615.70,1499.54,0.98,0.08,0.06


In [36]:
boosted_dt = ensemble_model(dt, method = 'Boosting')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,766.2829,1627000.8555,1275.5394,0.9821,0.0877,0.0652
1,915.4680,6235547.3207,2497.1078,0.9522,0.0890,0.0651
2,833.5151,2391858.4705,1546.5634,0.9781,0.0871,0.0656
3,863.9999,2277412.4576,1509.1098,0.9749,0.0846,0.0652
4,937.2337,7889268.2893,2808.7841,0.9016,0.1043,0.0714
5,971.6545,6866651.9147,2620.4297,0.9414,0.0990,0.0696
6,837.5502,2982929.1147,1727.1158,0.9720,0.0866,0.0635
7,824.1093,2511814.7814,1584.8706,0.9682,0.0867,0.0659
8,858.8770,3453781.8435,1858.4353,0.9661,0.0922,0.0653


In [39]:
final_dt = finalize_model(dt)

In [40]:
print(final_dt)

Pipeline(memory=FastMemory(location=C:\Users\user\AppData\Local\Temp\joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Carat Weight'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['Cut', 'Color', 'Clarity',
                                             'Polish', 'Symmetry', 'Report'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('ordinal_enc...
                 TransformerWrapper(include=['Cut', 'Color', 'Clarity',
                                             'Polish', 'Symmetry'],
                                    transformer=OneHotEncoder(cols=['Cut',
                                                                    'Color',
                                                                    'Clarity',
                                                                    'P

In [41]:
predict_model(final_dt)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Decision Tree Regressor,222.3417,665984.5830,816.0788,0.9939,0.0329,0.0136


,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price,prediction_label
3172,1.16,Ideal,E,SI1,EX,EX,GIA,7059,7059.0
658,1.14,Ideal,G,SI1,EX,VG,GIA,5809,5809.0
3180,1.25,Very Good,G,IF,VG,VG,GIA,10641,10641.0
1340,0.79,Ideal,G,SI1,VG,EX,GIA,2871,2871.0
586,2.23,Very Good,H,VS2,VG,VG,GIA,21706,22629.0
...,...,...,...,...,...,...,...,...,...
790,1.22,Ideal,F,VVS1,EX,EX,GIA,12906,12906.0
4934,0.83,Ideal,E,SI1,G,EX,GIA,3235,3235.0
469,1.51,Very Good,I,VS2,VG,VG,GIA,9058,9118.0
2061,1.16,Ideal,I,SI1,EX,VG,GIA,5091,5091.0


In [42]:
unseen_predictions = predict_model(final_dt, data=data_unseen)
unseen_predictions.head()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Decision Tree Regressor,233.8307,806221.3295,897.8983,0.9930,0.0419,0.0170


,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price,prediction_label
0,1.21,Ideal,G,VVS1,EX,EX,GIA,11572,11572.000000
1,2.00,Ideal,I,SI1,EX,VG,GIA,16775,16775.000000
2,1.51,Good,F,SI1,VG,G,GIA,10429,10429.000000
3,0.90,Ideal,F,SI1,EX,EX,GIA,4523,4523.000000
4,1.01,Very Good,I,SI1,VG,VG,GIA,4375,4233.333333


In [43]:
save_model(final_dt, 'Final DT Model Nure')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=C:\Users\user\AppData\Local\Temp\joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Carat Weight'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['Cut', 'Color', 'Clarity',
                                              'Polish', 'Symmetry', 'Report'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('ordinal_enc...
                  TransformerWrapper(include=['Cut', 'Color', 'Clarity',
                                              'Polish', 'Symmetry'],
                                     transformer=OneHotEncoder(cols=['Cut',
                                                                     'Color',
                                                                     'Clarity',
                                                        